<a href="https://colab.research.google.com/github/meguerg/lab-4KNN/blob/main/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**K-Nearest Neighbors Algorithm**

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=7e4eb0d5e772fa39f0298cf5a4af835de5043b1f31a16ec929e158633a2278da
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
!pip install scikit-learn


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
from pyspark.sql.functions import *

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import functions as F
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np

In [ ]:
spark = SparkSession.builder.appName("KNNExample").getOrCreate()

In [ ]:
df = spark.read.csv("/content/train.csv", header=True, inferSchema=True)

In [ ]:
selected_columns = ["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"]
df = df.select(selected_columns)

In [ ]:
df.show()

+--------+------+------+----+-----+-----+-------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|
+--------+------+------+----+-----+-----+-------+
|       0|     3|  male|22.0|    1|    0|   7.25|
|       1|     1|female|38.0|    1|    0|71.2833|
|       1|     3|female|26.0|    0|    0|  7.925|
|       1|     1|female|35.0|    1|    0|   53.1|
|       0|     3|  male|35.0|    0|    0|   8.05|
|       0|     3|  male|NULL|    0|    0| 8.4583|
|       0|     1|  male|54.0|    0|    0|51.8625|
|       0|     3|  male| 2.0|    3|    1| 21.075|
|       1|     3|female|27.0|    0|    2|11.1333|
|       1|     2|female|14.0|    1|    0|30.0708|
|       1|     3|female| 4.0|    1|    1|   16.7|
|       1|     1|female|58.0|    0|    0|  26.55|
|       0|     3|  male|20.0|    0|    0|   8.05|
|       0|     3|  male|39.0|    1|    5| 31.275|
|       0|     3|female|14.0|    0|    0| 7.8542|
|       1|     2|female|55.0|    0|    0|   16.0|
|       0|     3|  male| 2.0|    4|    1| 29.125|


In [ ]:
df = df.na.drop()

In [ ]:
df.show()

+--------+------+------+----+-----+-----+-------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|
+--------+------+------+----+-----+-----+-------+
|       0|     3|  male|22.0|    1|    0|   7.25|
|       1|     1|female|38.0|    1|    0|71.2833|
|       1|     3|female|26.0|    0|    0|  7.925|
|       1|     1|female|35.0|    1|    0|   53.1|
|       0|     3|  male|35.0|    0|    0|   8.05|
|       0|     1|  male|54.0|    0|    0|51.8625|
|       0|     3|  male| 2.0|    3|    1| 21.075|
|       1|     3|female|27.0|    0|    2|11.1333|
|       1|     2|female|14.0|    1|    0|30.0708|
|       1|     3|female| 4.0|    1|    1|   16.7|
|       1|     1|female|58.0|    0|    0|  26.55|
|       0|     3|  male|20.0|    0|    0|   8.05|
|       0|     3|  male|39.0|    1|    5| 31.275|
|       0|     3|female|14.0|    0|    0| 7.8542|
|       1|     2|female|55.0|    0|    0|   16.0|
|       0|     3|  male| 2.0|    4|    1| 29.125|
|       0|     3|female|31.0|    1|    0|   18.0|


In [ ]:
df = df.withColumn("Sex", F.when(df["Sex"] == "male", 0).otherwise(1))

In [ ]:
df.show()

+--------+------+---+----+-----+-----+-------+
|Survived|Pclass|Sex| Age|SibSp|Parch|   Fare|
+--------+------+---+----+-----+-----+-------+
|       0|     3|  0|22.0|    1|    0|   7.25|
|       1|     1|  1|38.0|    1|    0|71.2833|
|       1|     3|  1|26.0|    0|    0|  7.925|
|       1|     1|  1|35.0|    1|    0|   53.1|
|       0|     3|  0|35.0|    0|    0|   8.05|
|       0|     1|  0|54.0|    0|    0|51.8625|
|       0|     3|  0| 2.0|    3|    1| 21.075|
|       1|     3|  1|27.0|    0|    2|11.1333|
|       1|     2|  1|14.0|    1|    0|30.0708|
|       1|     3|  1| 4.0|    1|    1|   16.7|
|       1|     1|  1|58.0|    0|    0|  26.55|
|       0|     3|  0|20.0|    0|    0|   8.05|
|       0|     3|  0|39.0|    1|    5| 31.275|
|       0|     3|  1|14.0|    0|    0| 7.8542|
|       1|     2|  1|55.0|    0|    0|   16.0|
|       0|     3|  0| 2.0|    4|    1| 29.125|
|       0|     3|  1|31.0|    1|    0|   18.0|
|       0|     2|  0|35.0|    0|    0|   26.0|
|       1|   

In [ ]:
assembler = VectorAssembler(
    inputCols=["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"],
    outputCol="features"
)
df = assembler.transform(df)

In [ ]:
df=df.withColumn('Age',df['Age'].cast('Integer'))
df=df.withColumn('Fare',df['Fare'].cast('Integer'))

In [ ]:
df.show()

+--------+------+---+---+-----+-----+----+--------------------+
|Survived|Pclass|Sex|Age|SibSp|Parch|Fare|            features|
+--------+------+---+---+-----+-----+----+--------------------+
|       0|     3|  0| 22|    1|    0|   7|[3.0,0.0,22.0,1.0...|
|       1|     1|  1| 38|    1|    0|  71|[1.0,1.0,38.0,1.0...|
|       1|     3|  1| 26|    0|    0|   7|[3.0,1.0,26.0,0.0...|
|       1|     1|  1| 35|    1|    0|  53|[1.0,1.0,35.0,1.0...|
|       0|     3|  0| 35|    0|    0|   8|[3.0,0.0,35.0,0.0...|
|       0|     1|  0| 54|    0|    0|  51|[1.0,0.0,54.0,0.0...|
|       0|     3|  0|  2|    3|    1|  21|[3.0,0.0,2.0,3.0,...|
|       1|     3|  1| 27|    0|    2|  11|[3.0,1.0,27.0,0.0...|
|       1|     2|  1| 14|    1|    0|  30|[2.0,1.0,14.0,1.0...|
|       1|     3|  1|  4|    1|    1|  16|[3.0,1.0,4.0,1.0,...|
|       1|     1|  1| 58|    0|    0|  26|[1.0,1.0,58.0,0.0...|
|       0|     3|  0| 20|    0|    0|   8|[3.0,0.0,20.0,0.0...|
|       0|     3|  0| 39|    1|    5|  3

In [ ]:
mean_values = df.agg(*[F.mean(c).alias(c) for c in df.columns if c != "features"]).collect()[0]
stddev_values = df.agg(*[F.stddev(c).alias(c) for c in df.columns if c != "features"]).collect()[0]


In [ ]:
df.show()

+--------+------+---+---+-----+-----+----+--------------------+
|Survived|Pclass|Sex|Age|SibSp|Parch|Fare|            features|
+--------+------+---+---+-----+-----+----+--------------------+
|       0|     3|  0| 22|    1|    0|   7|[3.0,0.0,22.0,1.0...|
|       1|     1|  1| 38|    1|    0|  71|[1.0,1.0,38.0,1.0...|
|       1|     3|  1| 26|    0|    0|   7|[3.0,1.0,26.0,0.0...|
|       1|     1|  1| 35|    1|    0|  53|[1.0,1.0,35.0,1.0...|
|       0|     3|  0| 35|    0|    0|   8|[3.0,0.0,35.0,0.0...|
|       0|     1|  0| 54|    0|    0|  51|[1.0,0.0,54.0,0.0...|
|       0|     3|  0|  2|    3|    1|  21|[3.0,0.0,2.0,3.0,...|
|       1|     3|  1| 27|    0|    2|  11|[3.0,1.0,27.0,0.0...|
|       1|     2|  1| 14|    1|    0|  30|[2.0,1.0,14.0,1.0...|
|       1|     3|  1|  4|    1|    1|  16|[3.0,1.0,4.0,1.0,...|
|       1|     1|  1| 58|    0|    0|  26|[1.0,1.0,58.0,0.0...|
|       0|     3|  0| 20|    0|    0|   8|[3.0,0.0,20.0,0.0...|
|       0|     3|  0| 39|    1|    5|  3

In [ ]:
for col in df.columns:
    if col != "features":
        df = df.withColumn(col, (df[col] - mean_values[col]) / stddev_values[col])

In [ ]:
pandas_df = df.select("features", "Survived").toPandas()

In [ ]:
X = np.array([np.array(row) for row in pandas_df["features"]])
y = np.array(pandas_df["Survived"])

In [ ]:
knn_classifier = KNeighborsClassifier(n_neighbors=5)

In [ ]:
df = df.withColumn("Survived", df["Survived"].cast("integer"))

In [ ]:
pandas_df = df.select("features", "Survived").toPandas()

X = np.array([np.array(row) for row in pandas_df["features"]])
y = np.array(pandas_df["Survived"])

In [ ]:
knn_classifier = KNeighborsClassifier(n_neighbors=5)

cv_scores = cross_val_score(knn_classifier, X, y, cv=5, scoring="accuracy")

print("Cross-validation scores:", cv_scores)
print("Mean accuracy:", np.mean(cv_scores))

Cross-validation scores: [0.58041958 0.67832168 0.70629371 0.68531469 0.74647887]
Mean accuracy: 0.6793657047178174


In [ ]:
train, test = df.randomSplit([0.8, 0.2], seed=42)


In [ ]:
from pyspark.sql.types import DoubleType


predictions_df = spark.createDataFrame(
    zip(predictions.tolist(), test.select("Survived").toPandas()["Survived"].tolist()),
    ["prediction", "Survived"]
)


predictions_df = predictions_df.withColumn("prediction", predictions_df["prediction"].cast(DoubleType()))


evaluator = MulticlassClassificationEvaluator(labelCol="Survived")
accuracy = evaluator.evaluate(predictions_df, {evaluator.metricName: "accuracy"})


print("Accuracy on the test set:", accuracy)


Accuracy on the test set: 0.7719298245614035
